# Load Data

## hdbscan

In [1]:
import pickle

import pickle
file_path = "../18_rapids/result/20251112_044404/condensed_tree_object.pkl"
with open(file_path, 'rb') as f:
    clusterer = pickle.load(f)


In [13]:

Z, node_id_map = get_linkage_matrix_from_hdbscan(clusterer)

print(Z.shape)
print(Z[1])


len of sorted condensed tree: 344
len of linkage matrix: 172
344
Number of leaves: 173
Leaf ID Map Size: 173
current id: 173
Total Node ID Map Size: 345
current_id: 345
Max Lambda: 2.855132579803467
(172, 5)
[ 70.          71.         174.           0.17586493  70.        ]


## dummy data

In [12]:
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
import matplotlib.pyplot as plt
import plotly.graph_objects as go
# ダミーデータの準備 (5点)
X = np.array([
    [1.0, 1.0], [1.5, 1.5], [4.0, 4.0], [4.5, 4.5], [0.0, 5.0]
])
n_points = len(X)

# 連結行列 Z の生成 (ここではScipyを使用。このZを入力として使います)
# Z: [c1, c2, distance, count]
Z = linkage(X, method='single', metric='euclidean')

# Zの内容 (参考)
# print("連結行列 Z:\n", Z)
print(Z)

[[0.         1.         0.70710678 2.        ]
 [2.         3.         0.70710678 2.        ]
 [5.         6.         3.53553391 4.        ]
 [4.         7.         3.80788655 5.        ]]


# Dendrogram

## module

In [3]:

import sys

sys.setrecursionlimit(5000)
print(f"新しい制限: {sys.getrecursionlimit()}")

新しい制限: 5000


In [ ]:
import numpy as np
import hdbscan
from sklearn.datasets import make_blobs
from scipy.cluster.hierarchy import dendrogram
import plotly.graph_objects as go

def _get_leaves(condensed_tree):
    cluster_tree = condensed_tree[condensed_tree['child_size'] > 1]
    print(len(cluster_tree))
    if cluster_tree.shape[0] == 0:
        # Return the only cluster, the root
        return [condensed_tree['parent'].min()]

    root = cluster_tree['parent'].min()
    return _recurse_leaf_dfs(cluster_tree, root)
  
def _recurse_leaf_dfs(cluster_tree, current_node):
  children = cluster_tree[cluster_tree['parent'] == current_node]['child']
  if len(children) == 0:
      return [current_node,]
  else:
      return sum([_recurse_leaf_dfs(cluster_tree, child) for child in children], [])
  
def get_leaves(cluster_tree):
    """
    cluster_tree: (u, v, lambda_val, child_size, parent)
    """
    root = cluster_tree[:, 2].max()
    print(f"root: {root}")
    return recurse_leaf_dfs(cluster_tree, root)
    

def recurse_leaf_dfs(cluster_tree, current_node):
    # print(f"Visiting Node: {current_node}")
    child1 = cluster_tree[cluster_tree[:,2] == current_node][:,0]
    child2 = cluster_tree[cluster_tree[:,2] == current_node][:,1]
    # print(f"Children of Node {current_node}: Child1 {child1}, Child2 {child2}")

    if len(child1) == 0 and len(child2) == 0:
        
        return [current_node,]
    else:
        return sum([recurse_leaf_dfs(cluster_tree, child) for child in np.concatenate((child1, child2))], [])
    


def get_linkage_matrix_from_hdbscan(condensed_tree):
    """
    (child1, child2, parent, lambda_val, count)
    """
    linkage_matrix = []
    raw_tree = condensed_tree._raw_tree
    condensed_tree = condensed_tree.to_pandas()
    cluster_tree = condensed_tree[condensed_tree['child_size'] > 1]
    sorted_condensed_tree = cluster_tree.sort_values(by=['lambda_val','parent'], ascending=True)
    print(f"len of sorted condensed tree: {len(sorted_condensed_tree)}")

    for i in range(0, len(sorted_condensed_tree), 2):
    
        # 偶数行（i）と次の奇数行（i+1）をペアとして取得
        if i + 1 < len(sorted_condensed_tree):
            
            row_a = sorted_condensed_tree.iloc[i]
            row_b = sorted_condensed_tree.iloc[i+1]
            
            # **前提チェック**: lambda_valが同じであることを確認
            if row_a['lambda_val'] != row_b['lambda_val']:
                # lambda_valが異なる場合は、次の処理に進む（結合の前提が崩れる）
                raise ValueError(f"Lambda value mismatch at rows {i} and {i+1}: {row_a['lambda_val']} vs {row_b['lambda_val']}")
                
            # Parent IDが同じであることを確認 (同じ結合の結果である可能性が高い)
            if row_a['parent'] != row_b['parent']:
                # Parent IDが異なる場合は、このペアは単一の結合ではない可能性が高い
                raise ValueError(f"Parent ID mismatch at rows {i} and {i+1}: {row_a['parent']} vs {row_b['parent']}")
            
            child_a = row_a['child']
            child_b = row_b['child']
            lam = row_a['lambda_val']
            
            # count (サイズ) は、結合された2つの子ノードのサイズ合計を使うのが論理的だが、
            # HDBSCANは親ノードのサイズをリストで持っているため、ここではそのサイズを使用
            # より正確には、このParent IDを持つ全子ノードのサイズの合計を使うべきだが、
            # 2行の child_size の合計で暫定的に対応
            # total_size = row_a['child_size'] + row_b['child_size']


            total_size = raw_tree[raw_tree['child'] == row_a['parent']]['child_size']
            if len(total_size) == 0:
                total_size = row_a['child_size'] + row_b['child_size']
            else:
                total_size = total_size[0]
            # print(total_size)
            parent_id = row_a['parent']

            linkage_matrix.append([
                int(child_a), 
                int(child_b), 
                int(parent_id),
                lam, 
                total_size,
        ])   
    print(f"len of linkage matrix: {len(linkage_matrix)}")


    # 葉ノードに0-N-1のIDを振る
    node_id_map = {}
    current_id = 0
    leaves = _get_leaves(raw_tree)
    print(f"Number of leaves: {len(leaves)}")

    for leaf in leaves:
        node_id_map[int(leaf)] = current_id
        current_id += 1

    print(f"Leaf ID Map Size: {len(node_id_map)}")
    print(f"current id: {current_id}")

    # 結合ノードにIDを振る(linkage matrixのparent)
    for row in linkage_matrix.__reversed__():
        parent_id = row[2]
        if parent_id not in node_id_map:
            node_id_map[parent_id] = current_id
            current_id += 1

        else:
            print(f"Duplicate Parent ID found: {parent_id}")
            raise ValueError(f"Node ID {parent_id} already assigned!")
    print(f"Total Node ID Map Size: {len(node_id_map)}")
    print(f"current_id: {current_id}")

    # linkage matrixを書き換え
    max_lambda = max(row[3] for row in linkage_matrix)
    print(f"Max Lambda: {max_lambda}")
    linkage_matrix_mapped = [ 
        [node_id_map[row[0]], node_id_map[row[1]], node_id_map[row[2]], max_lambda - row[3], row[4]] 
        for row in linkage_matrix.__reversed__()
    ]

    return np.array(linkage_matrix_mapped), node_id_map # linkage matrix, parentid -> newid

# 

def calculate_strahler(Z_matrix: np.ndarray, n_leaves: int) -> np.ndarray:
    """
    Linkage Matrix (Z) に基づいて、各結合ノードのストラー数（Strahler Number）を計算する。
    
    Args:
        Z_matrix: Linkage Matrix (N-1 x 4のNumPy配列)。
        n_leaves: 元の観測値/葉ノードの数。
        
    Returns:
        np.ndarray: 各結合ノード（Zの各行）に対応するストラー数の配列。
    """
    n_merges = Z_matrix.shape[0]
    
    # 葉ノードのストラー数を初期化 (すべての葉ノードは S=1)
    # インデックス: 0から n_leaves - 1
    strahler_map = {i: 1 for i in range(n_leaves)}
    
    # Zの各行に対応するストラー数を格納するリスト
    merge_strahler_numbers = np.zeros(n_merges, dtype=int)
    
    # Z行列をボトムアップ（行 0 から N-2）で処理
    for i in range(n_merges):
        u_idx = int(Z_matrix[i, 0])  # 結合されるノード u
        v_idx = int(Z_matrix[i, 1])  # 結合されるノード v
        new_idx = n_leaves + i       # 新しく生成されるノード
        
        # 子ノードのストラー数を取得
        s_u = strahler_map.get(u_idx, 1)
        s_v = strahler_map.get(v_idx, 1)
        
        # ストラー数計算ロジック（二分木）
        if s_u == s_v:
            # S_u = S_v の場合、新しいノードのストラー数は S_u + 1
            s_new = s_u + 1
        else:
            # S_u != S_v の場合、新しいノードのストラー数は Max(S_u, S_v)
            s_new = max(s_u, s_v)
        
        # 結果を記録し、マップを更新
        merge_strahler_numbers[i] = s_new
        strahler_map[new_idx] = s_new

    return merge_strahler_numbers

def filter_linkage_matrix_by_strahler(Z_matrix: np.ndarray, S_min: int, N_leaves: int) -> np.ndarray:
    """
    Linkage Matrix (Z) にストラー数 (Strahler Number) を計算し、指定された最小ストラー数以上の結合のみを保持する。
    
    Args:
        Z_matrix (np.ndarray): Linkage Matrix (N-1 x 4)。
        S_min (int): フィルタリングのための最小ストラー数。

    Returns:
        np.ndarray: フィルタリングされたLinkage Matrix。
    """
    # 葉ノード数 (N_obs = Z.shape[0] + 1)
    # N_leaves = len(leaves) 

    # 1. ストラー数 (Strahler Number) の計算
    strahler_numbers = calculate_strahler(Z_matrix, N_leaves)

    # 2. Z_matrix の拡張 (ストラー数を5列目に追加)
    # (u, v, distance, count, strahler)
    Z_with_strahler = np.hstack((Z_matrix, strahler_numbers[:, np.newaxis]))

    # 3. フィルタリングの実行 (例: ストラー数 2 以上)
    # 5列目（インデックス4）がフィルタリング基準
    # check
    print(Z_with_strahler[1, :])
    filtered_Z_by_strahler = Z_with_strahler[Z_with_strahler[:, 5] >= S_min]

    # ストラー数の分布
    unique_strahler, counts = np.unique(strahler_numbers, return_counts=True)
    print("\n--- ストラー数の分布 ---")
    for s_num, count in zip(unique_strahler, counts):
        print(f"ストラー数 {int(s_num)}: {count} 本の枝")
    print("\n--- フィルタリング結果 ---")
    print(f"元のZ行列の行数: {Z_matrix.shape[0]}")
    print(f"ストラー数 >= {S_min} の行数: {filtered_Z_by_strahler.shape[0]}")


    # ----------------------------------------------------
    #

    # インデックスのマッピング

    node_id_map = {}
    current_id = 0
    leaves = get_leaves(filtered_Z_by_strahler)
    print(f"Leaves: {len(leaves)}")

    for leaf in leaves:
        node_id_map[int(leaf)] = current_id
        current_id += 1
    print(f"Number of leaves: {len(leaves)}")

    for row in filtered_Z_by_strahler:
        parent_id = row[2]
        if parent_id not in node_id_map:
            node_id_map[parent_id] = current_id
            current_id += 1
    print(f"Total Node ID Map Size: {len(node_id_map)}")
    print(f"current_id: {current_id}")

    linkage_matrix_mapped_strahler = [ 
        [node_id_map[row[0]], node_id_map[row[1]], node_id_map[row[2]], row[3], strahler_numbers[i]] 
        for i, row in enumerate(filtered_Z_by_strahler)
    ]
    return np.array(linkage_matrix_mapped_strahler), node_id_map




In [6]:
def compute_dendrogram_coords(Z, n_points):
    """
    連結行列 Zからデンドログラムの座標を計算します。

    Args:
        Z (numpy.ndarray): 連結行列 (n_points-1 x 4)。
        n_points (int): 元のデータ点数。

    Returns:
        tuple: (icoord, dcoord, leaf_order)
               icoord: X座標リスト (描画用)
               dcoord: Y座標リスト (描画用)
               leaf_order: 葉のインデックス順 (X軸の順序)
    """


    # --- 1. ノード情報の準備 ---
    # ノード総数 (葉 n_points + 内部ノード n_points-1)
    n_nodes = 2 * n_points - 1

    # ノード情報の初期化 (インデックス n_points-1 までが葉)
    # 葉ノード: x=None, y=0, size=1, left=None, right=None
    # 内部ノード: x=None, y=distance, size=count, left=c1, right=c2
    nodes = [{'x': None, 'y': 0.0, 'size': 1, 'left': None, 'right': None} for _ in range(n_points)]
    
    # 内部ノード情報をZから設定
    for i in range(n_points - 1):
        c1, c2, dist, count = Z[i]
        nodes.append({
            'x': None,
            'y': dist,
            'size': int(count),
            'left': int(c1),
            'right': int(c2)
        })

    # --- 2. X座標と葉の順序の再帰的計算 ---
    
    # ノードが持つ葉の順序リスト (インデックス) を返す
    def get_leaf_order(node_idx):
        node = nodes[node_idx]
        
        # ベースケース: 葉ノード
        if node_idx < n_points:
            return [node_idx] # 自身のインデックスを返す
        
        # 再帰ステップ: 内部ノード
        order_left = get_leaf_order(node['left'])
        order_right = get_leaf_order(node['right'])
        
        return order_left + order_right
    
    def get_leaf_order_sorted(node_idx):
        node = nodes[node_idx]
        
        if node_idx < n_points:
            return [node_idx]
        
        # 左右の子ノードのインデックスとサイズを取得
        C1_idx, C2_idx = node['left'], node['right']
   
        size_C1, size_C2 = nodes[C1_idx]['size'], nodes[C2_idx]['size']

        # --- 💡 ソート条件の適用 ---
        # 例: サイズの大きい方を左に配置（X軸の左側に来るように）
        if size_C1 < size_C2:
            # 子ノードのインデックスを入れ替える
            C1_idx, C2_idx = C2_idx, C1_idx
        
        # 入れ替えた順序で再帰呼び出し
        order_left = get_leaf_order_sorted(C1_idx)
        order_right = get_leaf_order_sorted(C2_idx)
        
        return order_left + order_right

    # X座標を決定する再帰関数
    def calculate_x_coord(node_idx, leaf_to_x):
        node = nodes[node_idx]
        
        # ベースケース: 葉ノード
        if node_idx < n_points:
            # 葉のX座標は、順序に基づいて 1, 3, 5, ... のように割り当てる
            x_coord = leaf_to_x[node_idx]
            node['x'] = x_coord
            return x_coord
        
        # 再帰ステップ: 内部ノード
        x_left = calculate_x_coord(node['left'], leaf_to_x)
        x_right = calculate_x_coord(node['right'], leaf_to_x)
        
        # 内部ノードのX座標は、子ノードのX座標の平均
        x_coord = (x_left + x_right) / 2.0
        node['x'] = x_coord
        return x_coord

    # まず葉の順序を決定 (最上位ノードからトラバース)
    root_node_idx = n_points - 1 + (n_points - 1) # 最後のZ行がルートノード
    leaf_order = get_leaf_order_sorted(root_node_idx)

    # 葉のX座標の割り当て (1, 3, 5, ...)
    leaf_to_x = {leaf_idx: 2 * i + 1 for i, leaf_idx in enumerate(leaf_order)}
    
    # X座標の計算をルートから実行
    calculate_x_coord(root_node_idx, leaf_to_x)


    # --- 3. 描画用線分 (icoord, dcoord) の生成 ---
    
    icoord = [] # X座標リスト [x1, x2, x3, x4]
    dcoord = [] # Y座標リスト [y1, y2, y3, y4]

    for i in range(n_points - 1):
        P = n_points + i # 親ノード P のインデックス
        C1 = nodes[P]['left']  # 左の子ノードのインデックス
        C2 = nodes[P]['right'] # 右の子ノードのインデックス

        # Y座標 (マージレベル)
        y_P = nodes[P]['y']
        y_C1 = nodes[C1]['y']
        y_C2 = nodes[C2]['y']

        # X座標
        x_P = nodes[P]['x']
        x_C1 = nodes[C1]['x']
        x_C2 = nodes[C2]['x']

        # 枝の線分の構成 (U字/Z字型)
        # 4つの点: (x_C1, y_C1) -> (x_C1, y_P) -> (x_C2, y_P) -> (x_C2, y_C2)
        # scipyのdendrogram出力形式に合わせるため、少し特殊な点の並びになります
        # 点1 (左の下端/マージ点): (x_C1, y_C1 または y_P)
        # 点2 (マージ点): (x_C1, y_P) または (x_P, y_P)
        # 点3 (マージ点): (x_C2, y_P) または (x_P, y_P)
        # 点4 (右の下端/マージ点): (x_C2, y_C2 または y_P)
        
        # ここでは、簡略化のため、線分を個別に作成します
        # 縦線: C1のX軸からy_C1->y_P
        # 横線: x_C1からx_C2、高さy_P
        # 縦線: C2のX軸からy_C2->y_P
        
        # scipyの出力形式に合わせて、4点の座標を格納
        # (x_C1, y_C1) -> (x_C1, y_P) -> (x_C2, y_P) -> (x_C2, y_C2)
        icoord.append([x_C1, x_C1, x_C2, x_C2])
        dcoord.append([y_C1, y_P, y_P, y_C2])

    return icoord, dcoord, leaf_order

In [7]:
def get_dendrogram_segments2(Z: np.ndarray):

    """
    Linkage Matrixからデンドログラム描画に必要な座標データを取得します。
    """
    # 実際には可視化しないが、座標計算のために呼び出す
    # no_plot=True を指定すると、描画はスキップされる
    n_points = Z.shape[0] + 1
    icoord, dcoord, leaf_order = compute_dendrogram_coords(Z, n_points)
    
    # 'icoord': 各結合の水平方向の座標 (x座標)
    # 'dcoord': 各結合の垂直方向の座標 (y座標、結合距離)
    # これらは描画するV字/逆U字のセグメントを定義します。
    
    segments = []
    
    # icoord, dcoord は (4, K) の配列で、Kは結合の数 (N-1)
    # 各 i は (x1, x2, x3, x4) の座標リスト
    for icoords, dcoords in zip(icoord, dcoord):
        # (x1, y1), (x2, y2), (x3, y3), (x4, y4) がセグメントの頂点
        x1, x2, x3, x4 = icoords
        y1, y2, y3, y4 = dcoords

    
        # if y1 == 0:
        #     y1 = y2
        #     new_x2 = (x2 + x3) / 2
        # if y4 == 0:
        #     y4 = y3
        #     new_x3 = (x2 + x3) / 2
        # if y1 !=0 and y4 !=0:
        #     new_x2 = x2
        #     new_x3 = x3
        

        # y1 = max(y1, Y_CUTOFF)
        # y4 = max(y4, Y_CUTOFF)
        # 1. 垂直線 (左の子ノードから結合点まで)
        segments.append([(x1, y1), (x2, y2)]) 
        # 2. 水平線 (結合したノード間)
        segments.append([(x2, y2), (x3, y3)]) 
        # 3. 垂直線 (右の子ノードから結合点まで)
        segments.append([(x4, y4), (x3, y3)]) 

    return segments

In [62]:
# def plot_dendrogram_plotly(segments, 
#                            colors=None, 
#                            scores=None, 
#                            is_selecteds=None,
#                            **kwargs):
#     fig = go.Figure()
    
#     for i, seg in enumerate(segments):
#         # seg は [[x_start, y_start], [x_end, y_end]]
#         x_coords = [seg[0][0], seg[1][0]]
#         y_coords = [seg[0][1], seg[1][1]]
        
#         color = 'blue' if colors is None else colors[i]
#         info = "no" if scores is None else f"{scores[i]:.2f}"
#         opacity = 0.2 if is_selecteds is not None and not is_selecteds[i] else 1.0
        
#         fig.add_trace(go.Scatter(
#             x=x_coords, 
#             y=y_coords, 
#             mode='lines',
#             line=dict(color=color, width=1),
#             # showlegend=True,
#             hoverinfo='text',
#             text=f'score={info})',
#             opacity=opacity
#         ))
    
#     fig.update_layout(
#         title='Simple Dendrogram Visualization',
#         xaxis_title='Observation Index',
#         yaxis_title='Distance / Height',
#         hovermode='closest',
        
#     )
#     # fig.show() # 実行環境によっては直接表示
    
#     # 葉ノードに名前を付ける場合は、dendrogram_data['leaves']と葉のy=0の座標を計算する必要があります。
#     fig.update_layout(height=800, width=1000)
#     # fig.update_traces(line =dict(color=colors, width=0.5))
#     return fig


import plotly.graph_objects as go
import typing

def plot_dendrogram_plotly(
    segments: typing.List, 
    colors: typing.Optional[typing.List] = None, 
    scores: typing.Optional[typing.List] = None, 
    is_selecteds: typing.Optional[typing.List] = None, 
    **kwargs
) -> go.Figure:
    """
    HDBSCANのデンドログラムをPlotlyで可視化し、**kwargsで渡された追加情報をホバーテキストに追加します。

    Args:
        segments (list): デンドログラムの各線分（セグメント）の座標リスト。
        colors (list, optional): 各セグメントの色リスト。
        scores (list, optional): 各セグメントに対応するスコア（例：Lambda/Stability）。
        is_selecteds (list, optional): 各セグメントが選択されているかどうかのブール値リスト。
        **kwargs: 各セグメントに対応する追加のホバー情報（キー：情報名、値：情報のリスト）。
                  例: cluster_size=[100, 50, ...], stability=[0.8, 0.9, ...]
    """
    fig = go.Figure()
    num_segments = len(segments)
    
    # kwargsに含まれる情報が有効かチェック
    # キー名とそれに対応するリストをタプルとして保持
    additional_data = []
    for key, value_list in kwargs.items():
        if isinstance(value_list, list) and len(value_list) == num_segments:
            additional_data.append((key, value_list))
        else:
            print(f"⚠️ Warning: Kwarg '{key}' must be a list of length {num_segments} to be included in hover data.")


    for i, seg in enumerate(segments):
        # seg は [[x_start, y_start], [x_end, y_end]]
        x_coords = [seg[0][0], seg[1][0]]
        y_coords = [seg[0][1], seg[1][1]]
        
        color = 'blue' if colors is None else colors[i]
        
        # --- ホバーテキストの構築 ---
        hover_lines = []
        
        # 1. スコア情報の追加
        if scores is not None and i < len(scores):
            hover_lines.append(f"Score: {scores[i]:.4f}")
        else:
            hover_lines.append("Score: N/A")
            
        # 2. **kwargsで渡された追加データの追加
        for key, value_list in additional_data:
            # key（情報名）と value_list[i]（そのセグメントの値）を整形して追加
            value = value_list[i]
            
            # 数値の場合は小数点以下を制御、その他はそのまま
            if isinstance(value, (int, float)):
                # 例: Size: 100, Stability: 0.92
                hover_lines.append(f"{key.replace('_', ' ').title()}: {value:.2f}") 
            else:
                # 例: Cluster ID: 5
                hover_lines.append(f"{key.replace('_', ' ').title()}: {value}")
        
        # 改行タグ <br> を使ってテキスト全体を結合
        full_hover_text = '<br>'.join(hover_lines)
        # ----------------------------
        
        opacity = 0.2 if is_selecteds is not None and not is_selecteds[i] else 1.0

        hover_mode = 'text'
        if is_selecteds is not None and not is_selecteds[i]:
            hover_mode = 'skip'
        
        fig.add_trace(go.Scatter(
            x=x_coords, 
            y=y_coords, 
            mode='lines',
            line=dict(color=color, width=1),
            showlegend=False,
            hoverinfo=hover_mode,
            text=full_hover_text, 
            opacity=opacity
        ))
    
    fig.update_layout(
        title='Simple Dendrogram Visualization',
        xaxis_title='Observation Index',
        yaxis_title='Distance / Height',
        hovermode='closest',
    )
    
    fig.update_layout(height=800, width=1000)
    return fig

## デンドログラムの描画

In [9]:
print(len(compute_dendrogram_coords(Z[:, [0, 1, 3, 4]], len(Z) + 1)[0]))

IndexError: index 4 is out of bounds for axis 1 with size 4

## ソートしたデンドログラムの描画

In [10]:
# icoord, dcoord, leaf_order = compute_dendrogram_coords(Z[:, [0, 1, 3, 4]], len(Z) + 1)
segments = get_dendrogram_segments2(Z[:, [0, 1, 3, 4]])
fig = plot_dendrogram_plotly(segments).show()


IndexError: index 4 is out of bounds for axis 1 with size 4

In [19]:
icoord_scratch, dcoord_scratch, leaf_order = compute_dendrogram_coords(Z, n_points)

print("\n--- スクラッチ計算結果 ---")
print("葉の順序 (leaf_order):", leaf_order)

# 5. 描画
fig, ax = plt.subplots(figsize=(6, 4))
ax.set_title("Dendrogram (Scratch Implementation)")

# 各枝を描画
for xs, ys in zip(icoord_scratch, dcoord_scratch):
    ax.plot(xs, ys, 'b-')

# X軸の目盛りとラベルを設定
# X座標は 1, 3, 5, ... となるため、目盛りは 1, 3, 5, ...
leaf_x_coords = [2 * i + 1 for i in range(n_points)]
ax.set_xticks(leaf_x_coords)
ax.set_xticklabels([f"Point {i}" for i in leaf_order])
ax.set_ylabel('Distance (Linkage)')
ax.set_xlabel('Data Points')
ax.grid(axis='y', linestyle='--', alpha=0.5)

NameError: name 'n_points' is not defined

## フィルタリングしたデンドログラムの描画

### strahler

In [20]:
_Z, node_id_map = filter_linkage_matrix_by_strahler(Z, S_min=4, N_leaves=len(_get_leaves(clusterer._raw_tree)))
segments = get_dendrogram_segments2(_Z[:, [0, 1, 3, 4]])
fig = plot_dendrogram_plotly(segments).show()

344
[ 70.          71.         174.           0.17586493  70.
   2.        ]

--- ストラー数の分布 ---
ストラー数 2: 25 本の枝
ストラー数 3: 34 本の枝
ストラー数 4: 113 本の枝

--- フィルタリング結果 ---
元のZ行列の行数: 172
ストラー数 >= 4 の行数: 113
root: 344.0
Leaves: 114
Number of leaves: 114
Total Node ID Map Size: 227
current_id: 227


### Stability

In [15]:
def compute_stability_python(condensed_tree):

    # 1. 最小クラスタとクラスタ数を定義 (Cythonと同じロジック)
    smallest_cluster = condensed_tree['parent'].min()
    num_clusters = condensed_tree['parent'].max() - smallest_cluster + 1
    
    largest_child = max(condensed_tree['child'].max(), smallest_cluster)

    # 2. lambda_birth の計算 (クラスタの誕生時の最小 lambda)
    # condensed_tree を 'child' でソート
    sorted_child_data = np.sort(condensed_tree[['child', 'lambda_val']], axis=0)
    
    # births_arr は、child ID に対応する lambda_birth を保持する
    births_arr = np.nan * np.ones(largest_child + 1, dtype=np.double)
    
    current_child = -1
    min_lambda = 0

    # NumPyの structured array を Pythonループで処理 (Cythonの loopを模倣)
    for row in range(sorted_child_data.shape[0]):
        child = sorted_child_data[row]['child']
        lambda_ = sorted_child_data[row]['lambda_val']

        if child == current_child:
            min_lambda = min(min_lambda, lambda_)
        elif current_child != -1:
            births_arr[current_child] = min_lambda
            current_child = child
            min_lambda = lambda_
        else:
            # Initialize
            current_child = child
            min_lambda = lambda_

    if current_child != -1:
        births_arr[current_child] = min_lambda
        
    births_arr[smallest_cluster] = 0.0 # ルートクラスタの lambda_birth は 0
    
    # 3. Stability スコアの計算
    
    # NumPyのベクトル演算で高速化可能だが、Cythonを模倣しループで計算
    result_arr = np.zeros(num_clusters, dtype=np.double)
    
    parents = condensed_tree['parent']
    sizes = condensed_tree['child_size']
    lambdas = condensed_tree['lambda_val']

    for i in range(condensed_tree.shape[0]):
        parent = parents[i]
        lambda_ = lambdas[i]
        child_size = sizes[i]
        result_index = parent - smallest_cluster
        
        # Stability(C) = Σ (lambda_death - lambda_birth) * size
        # condensed_treeの各行は、parent が child_size のクラスタを 'lambda_' で吸収/結合するステップを示す
        # この lambda_ は、HDBSCANロジックでは lambda_death と見なされる
        
        lambda_birth = births_arr[parent]
        
        # NOTE: HDBSCANのStability定義は複雑なため、ここはHDBSCANの内部ロジックを正確に模倣する必要があります。
        # オリジナルのCythonコードを再現:
        result_arr[result_index] += (lambda_ - lambda_birth) * child_size
        
    # 4. ID とスコアを辞書に変換
    node_ids = np.arange(smallest_cluster, condensed_tree['parent'].max() + 1)
    result_pre_dict = np.vstack((node_ids, result_arr)).T

    # フィルタリングされていないノードを含むため、dictに変換してIDとスコアを対応させる
    # [ID, Score] のペアの配列を辞書に変換
    return dict(zip(result_pre_dict[:, 0].astype(int), result_pre_dict[:, 1]))

In [22]:
stability_dict = compute_stability_python(clusterer._raw_tree)

In [23]:

node_id_map1_reversed = {v: k for k, v in node_id_map.items()}
scores = [stability_dict[node_id_map1_reversed[int(row[2])]] for row in Z] # parent ID -> stability score

def filter_linkage_matrix_by_score(Z, scores, threshold, n_points):
    """
    Z行列とスコアに基づき、フィルタリングされた枝とその子孫をマークし、
    フィルタされたノードを除いた情報でデンドログラム座標計算の準備をする。

    Args:
        Z (np.ndarray): 連結行列 (N-1 x 4)。
        scores (list/np.ndarray): Zの各行に対応するスコア (N-1)。
        threshold (float): スコアのフィルタリング閾値。
        n_points (int): 元のデータ点数。
    """
    n_nodes = 2 * n_points - 1
    
    # ノード情報の初期化
    # 葉ノード (0～n_points-1): 常にフィルタなし、スコアなし
    nodes = [{'is_filtered': False, 'score': None, 'children': []} for _ in range(n_points)]
    
    # 内部ノード (n_points～2*n_points-2): スコアとフィルタ状態を設定
    for i in range(n_points - 1):
        c1, c2, dist, count = Z[i]
        node_idx = n_points + i
        
        # フィルタリング条件: スコアが閾値を下回る
        is_filtered = (scores[i] < threshold)
        
        nodes.append({
            'is_filtered': is_filtered,
            'score': scores[i],
            'left': int(c1),
            'right': int(c2),
            # 簡略化のため、ここではleft/rightを使用
            'Z_row_index': i # Z行列の元のインデックス
        })
        
        # 既に作成された子ノードにも親子関係を逆向きに記録しておくと便利だが、
        # ここでは再帰で順方向 (親→子) に伝播させることに集中する。
    
    return nodes

def propagate_filter_state(nodes, node_idx):
    """
    指定されたノードがフィルタされている場合、その子孫ノードもフィルタ済みにマークする。
    """
    node = nodes[node_idx]
    
    # ノード自体が既にフィルタされていれば、子孫にも伝播させる
    if node['is_filtered'] == True:
        
        # ベースケース: 葉ノード (伝播終了)
        if node_idx < n_points:
            return
            
        # 再帰ステップ: 内部ノード
        # 左右の子ノードが存在する場合、子ノードもフィルタ済みにマークし、再帰呼び出し
        
        # 左の子ノードの処理
        left_idx = node['left']
        if left_idx < len(nodes): # 存在チェック
             # 子ノードをフィルタ済みに上書き
            nodes[left_idx]['is_filtered'] = True 
            propagate_filter_state(nodes, left_idx)
        
        # 右の子ノードの処理
        right_idx = node['right']
        if right_idx < len(nodes): # 存在チェック
             # 子ノードをフィルタ済みに上書き
            nodes[right_idx]['is_filtered'] = True
            propagate_filter_state(nodes, right_idx)

# ルートノードから伝播を開始 (最後のマージ操作がルートノードに対応)
root_node_idx = len(nodes) - 1


def get_filtered_z_and_map(Z, nodes, n_points):
    """
    フィルタされていないマージ操作のみを含む新しい連結行列Z'を作成し、
    デンドログラム描画に必要な情報（IDマッピング）を準備する。
    """
    # フィルタされていないノードに対応するZの行を取得
    unfiltered_Z_rows = []
    
    # 葉ノード（データ点）は常に存在するため、ここでは内部ノード（Zの行）をチェック
    for i in range(n_points - 1):
        node_idx = n_points + i
        if not nodes[node_idx]['is_filtered']:
            unfiltered_Z_rows.append(Z[i])
            
    # フィルタ後の新しいZ'（連結行列）
    if not unfiltered_Z_rows:
        # すべてフィルタされた場合、空の行列を返す
        return np.array([]), None
        
    Z_prime = np.array(unfiltered_Z_rows)
    
    # Z_primeの行数は元のN-1よりも小さくなる

    # フィルタされた枝のデンドログラム描画をどうするかは、設計上の選択になります:
    # 1. フィルタされた枝を「ノイズ」として扱う
    # 2. フィルタされたノードを、その親ノードのY座標で「打ち切り」として描画する
    
    # 💡 重要な点: Z'はもはや連結行列として完全な木構造を表現していない
    # そのため、Z'をそのまま座標計算関数に渡すのではなく、
    # フィルタされたノードの情報を利用して描画ロジックを修正する必要があります。

    return Z_prime


KeyError: 291.0

### dash

In [ ]:
import dash
from jupyter_dash import JupyterDash 
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import numpy as np
import pandas as pd

# データの準備（最大100点）
max_strahler = 4
max_stability = 100


# --- Dash アプリケーションの定義 ---
app = JupyterDash(__name__) 

app.layout = html.Div([
    html.H2("動的散布図ダッシュボード"),
    
    html.Div([
        html.Label("表示するデータポイント数:"),
        # スライダーで表示するデータ量を制御
        dcc.Slider(
            id='strahler-slider',
            min=0,
            max=max_strahler,
            step=1,
            value=0, # 初期値は50
            # marks={i: str(i) for i in range(10, max_strahler + 1, 10)},
            tooltip={"placement": "bottom", "always_visible": True}
        ),
        dcc.Slider(
            id='stability-slider',
            min=0,
            max=max_stability,
            step=5,
            value=20, # 初期値は20
            marks={i: str(i) for i in range(0, 101, 10)},
            tooltip={"placement": "bottom", "always_visible": True}
        )
    ], style={'padding': '20px'}),
    
    # dcc.Graph(id='scatter-output'),
    dcc.Graph(id='dendrogram-output')
])

# --- コールバック（グラフ更新ロジック）の定義 ---
@app.callback(
    Output('dendrogram-output', 'figure'),
    [Input('strahler-slider', 'value'), Input('stability-slider', 'value')]
)
def update_scatter_chart(strahler_threshold, stability_threshold):
    # フィルタリングされたZ行列とIDマップを取得
    _Z, node_id_map = filter_linkage_matrix_by_strahler(Z, S_min=strahler_threshold, N_leaves=len(_get_leaves(clusterer._raw_tree)))
    print(f"Filtered Z shape: {_Z.shape}")
    segments = get_dendrogram_segments2(_Z[:, [0, 1, 3, 4]])
    fig = plot_dendrogram_plotly(segments)
    
    return fig

# --- アプリケーションの実行 ---
# 'inline' モードでJupyter Notebook/Labのセル内に表示
app.run_server(mode='external', port=8055, host ='localhost')

Dash app running on http://localhost:8055/


344
[ 70.          71.         174.           0.17586493  70.
   2.        ]

--- ストラー数の分布 ---
ストラー数 2: 25 本の枝
ストラー数 3: 34 本の枝
ストラー数 4: 113 本の枝

--- フィルタリング結果 ---
元のZ行列の行数: 172
ストラー数 >= 0 の行数: 172
root: 344.0
Leaves: 173
Number of leaves: 173
Total Node ID Map Size: 345
current_id: 345
Filtered Z shape: (172, 5)
344
[ 70.          71.         174.           0.17586493  70.
   2.        ]

--- ストラー数の分布 ---
ストラー数 2: 25 本の枝
ストラー数 3: 34 本の枝
ストラー数 4: 113 本の枝

--- フィルタリング結果 ---
元のZ行列の行数: 172
ストラー数 >= 1 の行数: 172
root: 344.0
Leaves: 173
Number of leaves: 173
Total Node ID Map Size: 345
current_id: 345
Filtered Z shape: (172, 5)
344
[ 70.          71.         174.           0.17586493  70.
   2.        ]

--- ストラー数の分布 ---
ストラー数 2: 25 本の枝
ストラー数 3: 34 本の枝
ストラー数 4: 113 本の枝

--- フィルタリング結果 ---
元のZ行列の行数: 172
ストラー数 >= 2 の行数: 172
root: 344.0
Leaves: 173
Number of leaves: 173
Total Node ID Map Size: 345
current_id: 345
Filtered Z shape: (172, 5)
344
[ 70.          71.         174.           0.17586493  

## リーフを取得する

In [31]:
def _recurse_leaf_dfs(cluster_tree, current_node):
  children = cluster_tree[cluster_tree['parent'] == current_node]['child']
  if len(children) == 0:
      return [current_node,]
  else:
      return sum([_recurse_leaf_dfs(cluster_tree, child) for child in children], [])

In [ ]:
leaves = _recurse_leaf_dfs(clusterer._raw_tree, target_node_id)
print(len(leaves))

In [32]:
# (new_id -> cluster_id)
node_id_map_reversed = {v: k for k, v in node_id_map.items()}


In [ ]:
# フィルタリング語
# (new_id -> cluster_id)を更新
_Z, node_id_map_filtered = filter_linkage_matrix_by_strahler(Z, S_min=4, N_leaves=len(_get_leaves(clusterer._raw_tree)))

_node_id_map = {node_id_map_filtered[k]: v for k, v in node_id_map_reversed.items() if k in node_id_map_filtered}
print(f"length of filtered node id map: {_node_id_map.__len__()}")

print(f"length of Z: {len(_Z)}")
child_ids = set()
for row in _Z:
    child_ids.add(int(row[0]))
    child_ids.add(int(row[1]))

# idとmapから元のidを取得
original_ids = [_node_id_map[i] for i in range(len(child_ids))]
print(f"length of original ids: {len(original_ids)}")
print(original_ids[:10])


344
[ 70.          71.         174.           0.17586493  70.
   2.        ]

--- ストラー数の分布 ---
ストラー数 2: 25 本の枝
ストラー数 3: 34 本の枝
ストラー数 4: 113 本の枝

--- フィルタリング結果 ---
元のZ行列の行数: 172
ストラー数 >= 4 の行数: 113
root: 344.0
Leaves: 114
Number of leaves: 114
Total Node ID Map Size: 227
current_id: 227
length of filtered node id map: 227
length of Z: 113
length of original ids: 226
[100003, 100006, 100008, 100010, 100015, 100026, 100028, 100030, 100036, 100038]


In [25]:
raw_tree = clusterer._raw_tree

In [26]:
# もとのid内の点を取得 (raw_treeからcluster_id->point_idsの辞書を作成)
id_point_map = {id: _recurse_leaf_dfs(raw_tree, id) for id in node_id_map.keys()}

In [56]:
# condensed_tree._raw_treeからpoint_id_mapを作成
point_id_map = {}
# child_size=1の行のid -> parentのidの辞書
leaf_rows = raw_tree[raw_tree['child_size'] == 1]
point_id_map = {int(row['child']): int(row['parent']) for row in leaf_rows}


In [57]:
len(point_id_map)

100000

In [27]:
# フィルタ後のクラスタidに対応する点を取得
points = sum([id_point_map[_node_id_map[i]] for i in child_ids], [])
print(f"length of points: {len(points)}")
print(points[:10])

NameError: name 'child_ids' is not defined

In [ ]:
# フィルタされたlinkage matrixから、childのidを取得
print(f"length of Z: {len(Z)}")
child_ids = set()
for row in Z:
    child_ids.add(int(row[0]))
    child_ids.add(int(row[1]))

# idとmapから元のidを取得
original_ids = [node_id_map_reversed[i] for i in range(len(child_ids))]
print(f"length of original ids: {len(original_ids)}")
print(original_ids[:10])

In [ ]:
target_node_id = 100302
cluster_size = raw_tree[raw_tree["child"] == target_node_id][0]['child_size']
print(f"ノードID {target_node_id} のクラスタサイズ: {cluster_size}")


ノードID 100302 のクラスタサイズ: 1345


In [ ]:
for target_node_id in range(100101, 100310):
    _recurse_leaf_dfs(clusterer._raw_tree, target_node_id)

# api

In [78]:
# ==========================================================
# セル 1: 必要なライブラリと設定のインポート
# ==========================================================
import requests
import json
import random
from typing import List
import time

# --- サーバー設定 ---
# FastAPIサーバーのURLとエンドポイント
API_URL = "http://127.0.0.1:8000/umap_subset" 

# サーバー側で定義されたデータセットの総点数 (rapids_gpu_api_extended.py の N_TOTAL に依存)
N_TOTAL_POINTS = 100000 

# --- UMAPリクエスト設定 ---
# サーバーにリクエストするサブセットの点数
N_SUBSET = 5000 
# UMAPの出力次元数
N_COMPONENTS = 2
# 近傍数
N_NEIGHBORS = 15
# 最小距離
MIN_DIST = 0.1

print(f"API URL: {API_URL}")
print(f"全データ点数 (サーバー側): {N_TOTAL_POINTS}")
print(f"リクエストする点数: {N_SUBSET}")

# ==========================================================
# セル 2: ID生成とリクエストの実行
# ==========================================================

def generate_random_ids(total_range: int, count: int) -> List[int]:
    """
    0からtotal_range-1の範囲から、count個のランダムな一意のIDを生成します。
    """
    if count > total_range:
        count = total_range
    return random.sample(range(total_range), count)

# 1. リクエスト用のランダムなIDリストを生成
id_list = generate_random_ids(N_TOTAL_POINTS, N_SUBSET)
    
# 2. リクエストボディを構築
payload = {
    "id_list": id_list,
    "n_components": N_COMPONENTS,
    "n_neighbors": N_NEIGHBORS,
    "min_dist": MIN_DIST,
    "metric": "euclidean"
}

# 3. POSTリクエストを送信
print("APIリクエストを送信中...")
start_time = time.time()
try:
    response = requests.post(API_URL, json=payload, timeout=120)
    response.raise_for_status() # HTTPエラーが発生した場合に例外を投げる
    
    # 4. レスポンスを処理
    result = response.json()
    end_time = time.time()
    
    print("\n--- サーバーからの応答 ---")
    print(f"HTTPステータス: {response.status_code}")
    print(f"データ抽出とUMAP計算が完了しました (合計処理時間: {end_time - start_time:.4f}秒)")
    print("-" * 30)
    print(f"サーバー処理時間 (GPU/cuML): {result.get('execution_time_sec'):.4f} 秒")
    print(f"リクエスト点数: {result.get('requested_ids_count')}点")
    print(f"実際のサブセット形状: {result.get('subset_shape')}")
    
    # 埋め込み結果の取得と形状確認
    embedding = result.get('embedding')
    if embedding:
        print(f"埋め込み結果の形状: ({len(embedding)}, {len(embedding[0])})")
        print("最初の5点の埋め込み結果:")
        for i in range(min(5, len(embedding))):
            print(f"  {embedding[i]}")
        
except requests.exceptions.RequestException as e:
    print(f"\n--- リクエストエラー ---")
    print(f"APIサーバーへの接続に失敗しました。FastAPIサーバーが {API_URL} で起動しているか確認してください。")
    print(f"エラー詳細: {e}")
    if 'response' in locals():
         print(f"サーバー応答: {response.text}")



fig = px.scatter(x=[point[0] for point in embedding], y=[point[1] for point in embedding],
                 title="UMAP Embedding from FastAPI Server",
                 labels={"x": "UMAP Dimension 1", "y": "UMAP Dimension 2"}
                 ).show()

API URL: http://127.0.0.1:8000/umap_subset
全データ点数 (サーバー側): 100000
リクエストする点数: 5000
APIリクエストを送信中...

--- サーバーからの応答 ---
HTTPステータス: 200
データ抽出とUMAP計算が完了しました (合計処理時間: 0.4859秒)
------------------------------
サーバー処理時間 (GPU/cuML): 0.3651 秒
リクエスト点数: 5000点
実際のサブセット形状: [5000, 300]
埋め込み結果の形状: (5000, 2)
最初の5点の埋め込み結果:
  [-6.26643180847168, 6.005413055419922]
  [9.6387939453125, 2.8907546997070312]
  [9.062829971313477, 4.098848342895508]
  [-4.732667922973633, 5.587291717529297]
  [10.224967002868652, 2.8732166290283203]


In [ ]:
# 

# Visual Mapping

In [66]:
Z, node_id_map = get_linkage_matrix_from_hdbscan(clusterer)
stability_dict = compute_stability_python(clusterer._raw_tree)

raw_tree = clusterer._raw_tree
leaf_rows = raw_tree[raw_tree['child_size'] == 1]
point_id_map = {int(row['child']): int(row['parent']) for row in leaf_rows}

data = np.load("../18_rapids/result/20251112_044404/embedding.npz")
embedding = data["embedding"]

len of sorted condensed tree: 344
len of linkage matrix: 172
344
Number of leaves: 173
Leaf ID Map Size: 173
current id: 173
Total Node ID Map Size: 345
current_id: 345
Max Lambda: 2.855132579803467


In [67]:
id_node_map = {v: k for k, v in node_id_map.items()}


In [71]:
# e+01とかいう表記を避けるための設定
np.set_printoptions(suppress=True, precision=4)
Z[:10, :]

array([[ 67.    ,  68.    , 173.    ,   0.    ,  87.    ],
       [ 70.    ,  71.    , 174.    ,   0.1759,  70.    ],
       [ 76.    ,  77.    , 175.    ,   0.3731, 218.    ],
       [ 66.    , 173.    , 176.    ,   0.4648, 185.    ],
       [ 65.    , 176.    , 177.    ,   0.5596, 228.    ],
       [ 73.    ,  74.    , 178.    ,   0.618 ,  13.    ],
       [ 62.    ,  63.    , 179.    ,   0.6236,  38.    ],
       [ 64.    , 177.    , 180.    ,   0.63  , 304.    ],
       [ 69.    , 174.    , 181.    ,   0.7063,  89.    ],
       [180.    , 181.    , 182.    ,   0.7123, 394.    ]])

344
[ 70.      71.     174.       0.1759  70.       2.    ]

--- ストラー数の分布 ---
ストラー数 2: 25 本の枝
ストラー数 3: 34 本の枝
ストラー数 4: 113 本の枝

--- フィルタリング結果 ---
元のZ行列の行数: 172
ストラー数 >= 0 の行数: 172
root: 344.0
Leaves: 173
Number of leaves: 173
Total Node ID Map Size: 345
current_id: 345
length of selected cluster ids: 11
length of selected new cluster ids: 11
selected new cluster ids: [45, 114, 46, 51, 53, 103, 48, 98, 49, 100]
len segment, is_selected: 516, 516
Selected cluster similarity matrix shape: (11, 11)
shape of selected cluster ids: 11
344
[ 70.      71.     174.       0.1759  70.       2.    ]

--- ストラー数の分布 ---
ストラー数 2: 25 本の枝
ストラー数 3: 34 本の枝
ストラー数 4: 113 本の枝

--- フィルタリング結果 ---
元のZ行列の行数: 172
ストラー数 >= 0 の行数: 172
root: 344.0
Leaves: 173
Number of leaves: 173
Total Node ID Map Size: 345
current_id: 345
len segment, is_selected: 516, 0
344
[ 70.      71.     174.       0.1759  70.       2.    ]

--- ストラー数の分布 ---
ストラー数 2: 25 本の枝
ストラー数 3: 34 本の枝
ストラー数 4: 113 本の枝

--- フィルタリング結果 ---
元のZ行列の行数: 172
ストラ

In [68]:
# デンドログラムの描画に追加する情報の設定
extra_info_kwargs = {
    "cluster parent" : [id_node_map[int(row[2])] for row in Z],
    "cluster child 1": [id_node_map[int(row[0])] for row in Z],
    "cluster child 2": [id_node_map[int(row[1])] for row in Z],
    "Cluster Size": [row[3] for row in Z],
    "Stability": [stability_dict[id_node_map[int(row[2])]] if id_node_map[int(row[2])] in stability_dict else 0.0 for row in Z]  
}

extra_info_kwargs["cluster parent"] = sum([extra_info_kwargs["cluster parent"]] * 3, [])
extra_info_kwargs["cluster child 1"] = sum([extra_info_kwargs["cluster child 1"]] * 3, [])
extra_info_kwargs["cluster child 2"] = sum([extra_info_kwargs["cluster child 2"]] * 3, [])
extra_info_kwargs["Cluster Size"] = sum([extra_info_kwargs["Cluster Size"]] * 3, [])
extra_info_kwargs["Stability"] = sum([extra_info_kwargs["Stability"]] * 3, [])

In [78]:
from typing import List
# pointのid列を受け取ってクラスタ集合を返す
def get_clusters_from_points(point_ids: List[int], point_id_map: dict) -> List[int]:
    cluster_ids = set()
    for pid in point_ids:
        if pid in point_id_map:
            cluster_ids.add(point_id_map[pid])
    return list(cluster_ids)

# リーフとなるクラスタのみを取得
leaves = _get_leaves(clusterer._raw_tree)




344


## dash

In [44]:
# 使用する変数いろいろ
# 高次元データ
n_samples = 100000
n_dimensions = 300
X, y = np.random.randn(n_samples, n_dimensions), np.random.randint(0, 10, size=n_samples)


In [19]:
# 適当な座標を返す
def random_embedding(n_points, n_components):
    return np.random.rand(n_points, n_components)
data = np.load("../18_rapids/result/20251112_044404/embedding.npz")
embedding = data["embedding"]
labels = np.load("../18_rapids/result/20251112_044404/data.npz")["labels"]


In [26]:
import numpy as np
from scipy.linalg import inv, det
import warnings
from sklearn.covariance import LedoitWolf
from typing import Dict
# --- 1. 分布のパラメータ推定 ---

def estimate_single_gaussian_params(X_data: np.ndarray) -> Dict[str, np.ndarray]:
    """
    単一の高次元データセットから、多変量正規分布の平均ベクトルと共分散行列を推定する。
    サンプル数が次元数より少ない場合、Ledoit-Wolf収縮法を用いて共分散行列を頑健に推定する。

    Args:
        X_data (np.ndarray): 単一のクラスタ（またはデータセット）に属するデータ (N, D)。

    Returns:
        Dict: {'mu': 平均ベクトル, 'Sigma': 共分散行列}
    """
    N, D = X_data.shape # サンプル数 N, 次元数 D

    if N == 0:
        raise ValueError("Input data array must not be empty.")
    
    # 1. 平均ベクトル (mu) の推定
    mu = np.mean(X_data, axis=0)

    # 2. 共分散行列 (Sigma) の推定
    if N == 1:
        # サンプル数が1の場合、共分散はゼロ
        warnings.warn("N=1. Covariance matrix is set to zero (plus regularization).")
        Sigma = np.zeros((D, D))
        
    elif N < D + 1:
        # N < D+1 の場合: 特異行列になるリスクが高いため、Ledoit-Wolf収縮推定を使用
        warnings.warn(f"N={N} < D+1={D+1}. Using Ledoit-Wolf shrinkage for robust covariance estimation.")
        
        # Ledoit-Wolf収縮推定器を初期化・学習
        lw = LedoitWolf()
        lw.fit(X_data)
        Sigma = lw.covariance_
        
    else:
        # N >= D+1 の場合: 標準的な最尤推定を使用
        Sigma = np.cov(X_data, rowvar=False)

    # 3. 最終的な正則化チェック
    # 収縮推定を行ってもなお条件数が悪い場合に、微小な値を加算
    if np.linalg.cond(Sigma) > 1e15: # より厳しい条件数でチェック
        warnings.warn("Covariance matrix highly ill-conditioned. Applying final small regularization.")
        Sigma += np.eye(D) * 1e-6
        
    return {'mu': mu, 'Sigma': Sigma}

# --- 2. 類似度測定 (非類似度) ---

def kl_divergence_gaussian(mu1, Sigma1, mu2, Sigma2) -> float:
    """
    多変量ガウス分布 N1 から N2 へのKL情報量 D_KL(N1 || N2) を計算する。
    """
    D = mu1.shape[0]

    try:
        Sigma2_inv = inv(Sigma2)
    except np.linalg.LinAlgError:
        warnings.warn("Sigma2 is singular. KL divergence is undefined.")
        return np.nan

    # 1. ログデターミナント項
    log_det_term = np.log(det(Sigma2) / det(Sigma1))

    # 2. トレース項
    trace_term = np.trace(Sigma2_inv @ Sigma1)

    # 3. マハラノビス距離項 (平均の違い)
    diff_mu = mu2 - mu1
    mahalanobis_term = diff_mu.T @ Sigma2_inv @ diff_mu

    kl_div = 0.5 * (log_det_term + trace_term + mahalanobis_term - D)

    return kl_div

# --- 3. 類似度測定 (重なり) ---

def bhattacharyya_coefficient_gaussian(mu1, Sigma1, mu2, Sigma2) -> float:
    """
    多変量ガウス分布間のバタチャリヤ係数 BC を計算する。
    """
    D = mu1.shape[0]
    
    # 共分散行列の平均 Sigma = 0.5 * (Sigma1 + Sigma2)
    Sigma = 0.5 * (Sigma1 + Sigma2)

    try:
        Sigma_inv = inv(Sigma)
    except np.linalg.LinAlgError:
        warnings.warn("Sigma (mean covariance) is singular. Bhattacharyya Coefficient is undefined.")
        return np.nan

    # 1. バタチャリヤ距離の平均項（マハラノビス距離の変形）
    diff_mu = mu1 - mu2
    db_mu_term = 0.125 * diff_mu.T @ Sigma_inv @ diff_mu

    # 2. バタチャリヤ距離の共分散項
    db_cov_term = 0.5 * np.log(det(Sigma) / np.sqrt(det(Sigma1) * det(Sigma2)))

    # バタチャリヤ距離
    db_distance = db_mu_term + db_cov_term
    
    # バタチャリヤ係数 = exp(-距離)
    bc = np.exp(-db_distance)

    return bc

# --- 4. マハラノビス距離 (参考) ---

def mahalanobis_distance(mu1, mu2, Sigma_pooled) -> float:
    """
    プールされた共分散行列 Sigma_pooled を用いて、2つの平均間のマハラノビス距離を計算する。
    """
    try:
        Sigma_inv = inv(Sigma_pooled)
    except np.linalg.LinAlgError:
        warnings.warn("Sigma_pooled is singular. Mahalanobis distance is undefined.")
        return np.nan
    
    diff_mu = mu1 - mu2
    # 距離の2乗
    dist_sq = diff_mu.T @ Sigma_inv @ diff_mu
    
    return np.sqrt(dist_sq)





In [85]:
sample_list = [1, 2, 3, 4, 5]
# 3倍に複製
expanded_list = sum([sample_list] * 3, [])
print(expanded_list)

[1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5]


In [24]:
# もとのid内の点を取得 (raw_treeからcluster_id->point_idsの辞書を作成)
id_point_map = {id: _recurse_leaf_dfs(raw_tree, id) for id in node_id_map.keys()}

In [32]:
# クラスタ間の類似度を事前に計算する(辞書で保存)

similarity_bc = {}
similarity_kl = {}
similarity_m = {}
for i, cluster_id1 in enumerate(node_id_map.keys()):
    # print(f"Computing similarities for cluster {i+1}/{len(node_id_map)} (ID: {cluster_id1})")
    point_ids1 = id_point_map[cluster_id1]
    X_data1 = embedding[point_ids1]
    params1 = estimate_single_gaussian_params(X_data1)
    
    for j, cluster_id2 in enumerate(node_id_map.keys()):
        if i >= j:
            continue # 対称性を利用して計算量削減
        
        point_ids2 = id_point_map[cluster_id2]
        X_data2 = embedding[point_ids2]
        params2 = estimate_single_gaussian_params(X_data2)
        
        # KLダイバージェンス
        kl_div = kl_divergence_gaussian(params1['mu'], params1['Sigma'],
                                        params2['mu'], params2['Sigma'])
        
        # バタチャリヤ係数
        bc = bhattacharyya_coefficient_gaussian(params1['mu'], params1['Sigma'],
                                                params2['mu'], params2['Sigma'])
        # マハラノビス距離
        pooled_cov = 0.5 * (params1['Sigma'] + params2['Sigma'])
        m_dist = mahalanobis_distance(params1['mu'], params2['mu'], pooled_cov)
        
        # 辞書に保存
        similarity_kl[(cluster_id1, cluster_id2)] = kl_div
        similarity_bc[(cluster_id1, cluster_id2)] = bc
        similarity_m[(cluster_id1, cluster_id2)] = m_dist   


344
[ 70.          71.         174.           0.17586493  70.
   2.        ]

--- ストラー数の分布 ---
ストラー数 2: 25 本の枝
ストラー数 3: 34 本の枝
ストラー数 4: 113 本の枝

--- フィルタリング結果 ---
元のZ行列の行数: 172
ストラー数 >= 0 の行数: 172
root: 344.0
Leaves: 173
Number of leaves: 173
Total Node ID Map Size: 345
current_id: 345
length of selected cluster ids: 35
length of selected new cluster ids: 35
selected new cluster ids: [21, 47, 144, 209, 93, 94, 54, 170, 55, 28]
len segment, is_selected: 516, 516
Selected cluster similarity matrix shape: (35, 35)
344
[ 70.          71.         174.           0.17586493  70.
   2.        ]

--- ストラー数の分布 ---
ストラー数 2: 25 本の枝
ストラー数 3: 34 本の枝
ストラー数 4: 113 本の枝

--- フィルタリング結果 ---
元のZ行列の行数: 172
ストラー数 >= 0 の行数: 172
root: 344.0
Leaves: 173
Number of leaves: 173
Total Node ID Map Size: 345
current_id: 345
length of selected cluster ids: 166
length of selected new cluster ids: 166
selected new cluster ids: [172, 0, 1, 2, 4, 170, 167, 168, 169, 166]
len segment, is_selected: 516, 516
Selected cluster

In [75]:
print(node_id_map[100302])


192


In [76]:
print(id_node_map[192])

100302


In [80]:
# import dash
from jupyter_dash import JupyterDash 
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import numpy as np
import pandas as pd

# データの準備（最大100点）
max_strahler = 4
max_stability = 100

df = pd.DataFrame({
    'x': embedding[:, 0],
    'y': embedding[:, 1],
    'label': labels
})




# --- Dash アプリケーションの定義 ---
app = JupyterDash(__name__) 

app_layout = html.Div([
    html.H2("動的散布図とデンドログラムダッシュボード", style={'textAlign': 'center', 'marginBottom': '20px'}),
    
    
    # 左右にプロットを配置するコンテナ
    html.Div([
        # --- 1. 左側の領域: 散布図 ---
        html.Div([
            html.H3("散布図 (Scatter Plot)", style={'textAlign': 'center'}),
            dcc.Graph(
                id='scatter-output',
                style={'height': '70vh'} # 高さを確保
            ),
        ], style={'flex': 1, 'minWidth': '48%', 'padding': '10px'}),
        
        # --- 2. 右側の領域: デンドログラム + スライダー ---
        html.Div([
            # スライダーコンテナ (コンパクトに配置)
            # デンドログラムの上に配置するため、まず定義
            html.Div([
                # Strahlter スライダー
                html.Div([
                    html.Label("Strahlter値 (データポイント数):"),
                    dcc.Slider(
                        id='strahler-slider',
                        min=0,
                        max=max_strahler,
                        step=1,
                        value=0,
                        # コンパクトなマーク表示
                        marks={i: str(i) for i in range(0, max_strahler + 1, 5) if i != 0}, 
                        tooltip={"placement": "bottom", "always_visible": True}
                    )
                ], style={'flex': 1, 'marginRight': '20px'}),

                # Stability スライダー
                html.Div([
                    html.Label("Stability (%) 閾値:"),
                    dcc.Slider(
                        id='stability-slider',
                        min=0,
                        max=max_stability,
                        step=5,
                        value=20,
                        marks={i: str(i) for i in range(0, 101, 20)},
                        tooltip={"placement": "bottom", "always_visible": True}
                    )
                ], style={'flex': 1}),

            ], style={
                'display': 'flex',        # スライダーを横並びにする
                'justifyContent': 'space-between',
                'padding': '10px 20px 20px',
                'borderBottom': '1px solid #ddd' # 区切り線
            }),

            html.H3("デンドログラム (Dendrogram)", style={'textAlign': 'center'}),
            dcc.Graph(
                id='dendrogram-output',
                style={'height': '70vh'} # 高さを確保
            ),
        ], style={'flex': 1, 'minWidth': '48%', 'padding': '10px'}),

    ], style={
        'display': 'flex',               # 散布図とデンドログラムを横並びにする
        'flexDirection': 'row',
        'gap': '1%',                     # グラフ間の隙間
        'alignItems': 'flex-start'       # コンテンツを上揃えにする
    }),
    html.Div(id="output", style={"marginTop": 20, "fontWeight": "bold", "color": "blue"}),
    html.Div(id="detail"),
    dcc.Graph(id='detail-output1', style={'height': '50vh', 'marginTop': '20px'}),
    dcc.Graph(id='detail-output2', style={'height': '50vh', 'marginTop': '20px'}),
    dcc.Graph(id='detail-output3', style={'height': '50vh', 'marginTop': '20px'})
])

app.layout = app_layout

@app.callback(
    Output('dendrogram-output', 'figure'),
    Output('output', 'children'),
    Output('detail-output1', 'figure'),
    Output('detail-output2', 'figure'),
    Output('detail-output3', 'figure'),
    [
        Input('strahler-slider', 'value'),
        Input('stability-slider', 'value'),
        Input("scatter-output", 'selectedData')
    ]
)
def update_dendrogram(strahler_threshold, stability_threshold, selectedData):
    # デフォルトのデンドログラム
    # _Z, _node_id_map = filter_linkage_matrix_by_strahler(
    #     Z, S_min=strahler_threshold, N_leaves=len(_get_leaves(clusterer._raw_tree))
    # )
    segments = get_dendrogram_segments2(Z[:, [0, 1, 3, 4]])
    selected_new_cluster_ids = []
    is_selected = []

    # 選択データがある場合
    if selectedData is not None:
      
        # 選択されたポイントのクラスタidを取得
        selected_cluster_ids = get_clusters_from_points(
            [int(point['pointIndex']) for index, point in enumerate(selectedData['points']) if point["customdata"][0] is not -1],
            point_id_map
        )
        print(f"length of selected cluster ids: {len(selected_cluster_ids)}")
        print(f"selected cluster ids: {selected_cluster_ids[:10]}")

        selected_new_cluster_ids = [node_id_map[k] for k in selected_cluster_ids if k in node_id_map]
        print(f"length of selected new cluster ids: {len(selected_new_cluster_ids)}")
        print(f"selected new cluster ids: {selected_new_cluster_ids[:10]}")

        # デンドログラム上のクラスタID
        # selected_new_cluster_ids = sum([[i]*3 for i in selected_new_cluster_ids], [])
        # childに含まれている枝を選択状態にする
        is_selected = [True if (row[0] in selected_new_cluster_ids) or (row[1] in selected_new_cluster_ids) else False for row in Z]
        # is_selected = [True if int(row[2]) in selected_new_cluster_ids else False for row in Z]
        
        text = f"選択されたデータポイントのクラスターid: {', '.join(map(str, selected_new_cluster_ids)) + '.'.join(map(str, selected_cluster_ids))}"
    else:
        text = "選択されたデータポイントはありません。"
    is_selected = sum([[i]*3 for i in is_selected], [])
    
    print(f"len segment, is_selected: {len(segments)}, {len(is_selected)}")



    
    fig = plot_dendrogram_plotly(segments, is_selecteds=is_selected, **extra_info_kwargs)

    # クラスタ(selected cluster ids)間の類似度を表示(similality_matrix_bc などを使用)
    selected_cluster_similarity_matrix1 = np.zeros((len(selected_cluster_ids), len(selected_cluster_ids)))
    selected_cluster_similarity_matrix2 = np.zeros((len(selected_cluster_ids), len(selected_cluster_ids)))
    selected_cluster_similarity_matrix3 = np.zeros((len(selected_cluster_ids), len(selected_cluster_ids)))
    for i, cid1 in enumerate(selected_cluster_ids):
        for j, cid2 in enumerate(selected_cluster_ids):
            if i == j:
                selected_cluster_similarity_matrix1[i, j] = 0.0
                selected_cluster_similarity_matrix2[i, j] = 0.0
                selected_cluster_similarity_matrix3[i, j] = 0.0
            elif i < j:
                key = (cid1, cid2) if (cid1, cid2) in similarity_m else (cid2, cid1)
                selected_cluster_similarity_matrix1[i, j] = similarity_m.get(key, 0)
                selected_cluster_similarity_matrix1[j, i] = selected_cluster_similarity_matrix1[i, j]
                selected_cluster_similarity_matrix2[i, j] = similarity_bc.get(key, 0)
                selected_cluster_similarity_matrix2[j, i] = selected_cluster_similarity_matrix2[i, j]
                selected_cluster_similarity_matrix3[i, j] = similarity_kl.get(key, 0)
                selected_cluster_similarity_matrix3[j, i] = selected_cluster_similarity_matrix3[i, j]
    print(f"Selected cluster similarity matrix shape: {selected_cluster_similarity_matrix1.shape}")
    print(f"shape of selected cluster ids: {len(selected_cluster_ids)}")

    fig_similarity1 = px.imshow(selected_cluster_similarity_matrix1,
                    x=selected_cluster_ids,
                    y=selected_cluster_ids,
                    labels=dict(x="Cluster ID", y="Cluster ID", color="Mahalanobis Distance"),
                    color_continuous_scale='Viridis',
                    ).update_xaxes(type='category').update_yaxes(type='category')
    fig_similarity2 = px.imshow(selected_cluster_similarity_matrix2,
                    x=selected_cluster_ids,
                    y=selected_cluster_ids,
                    labels=dict(x="Cluster ID", y="Cluster ID", color="KL Divergence"),
                    color_continuous_scale='Viridis',
                    ).update_xaxes(type='category').update_yaxes(type='category')
    fig_siilarity3 = px.imshow(selected_cluster_similarity_matrix3,
                    x=selected_cluster_ids,
                    y=selected_cluster_ids,
                    labels=dict(x="Cluster ID", y="Cluster ID", color="Bhattacharyya Coefficient"),
                    color_continuous_scale='Viridis',
                    ).update_xaxes(type='category').update_yaxes(type='category')



    return fig, text, fig_similarity1, fig_similarity2, fig_siilarity3

# デンドログラムの更新コールバック
# @app.callback(
#     Output('dendrogram-output', 'figure'),
#     [Input('strahler-slider', 'value'), Input('stability-slider', 'value')]
# )
# def update_scatter_chart(strahler_threshold, stability_threshold):
#     # フィルタリングされたZ行列とIDマップを取得
#     _Z, node_id_map = filter_linkage_matrix_by_strahler(Z, S_min=strahler_threshold, N_leaves=len(_get_leaves(clusterer._raw_tree)))
#     print(f"Filtered Z shape: {_Z.shape}")
#     segments = get_dendrogram_segments2(_Z[:, [0, 1, 3, 4]])
#     fig = plot_dendrogram_plotly(segments)
    
#     return fig

# 散布図の更新コールバック
@app.callback(
    Output('scatter-output', 'figure'),
    [Input('strahler-slider', 'value'), Input('stability-slider', 'value')]
)
def update_scatter_chart(strahler_threshold, stability_threshold):
    
    # apiからembeddingを取得
    # API_URL = "http://localhost:5000/api/umap_subset"
    # payload = {
    #     "id_list": leaves, # 次元削減するポイントのidリスト
    #     "n_components": N_COMPONENTS,
    #     "n_neighbors": N_NEIGHBORS,
    #     "min_dist": MIN_DIST,
    #     "metric": "euclidean"
    # }
    # response = requests.post(API_URL, json=payload, timeout=120)

    # embedding = response.json().get('embedding')
    # print(f"Embedding shape: ({len(embedding)}, {len(embedding[0])})")
    
    #####################################
    # embedding = random_embedding(n_samples, 2) # ダミー埋め込み

    fig = px.scatter(df,
                    x='x',
                    y='y',
                    custom_data=['label'],
                     )
    fig.update_traces(marker=dict(size=2))
    fig.update_layout(dragmode="lasso")

    return fig

# # 選択されたデータポイントを表示するコールバック
# @app.callback(
#     # Output('output', 'children'), 
#     Output('dendrogram-output', 'figure'),
#     [Input("scatter-output", 'selectedData'), Input('strahler-slider', 'value'), Input('stability-slider', 'value')]
# )
# def display_selected_data(selectedData, strahler_threshold, stability_threshold):
#     if selectedData is None:
#         return "選択されたデータポイントはありません。"
    
#     # 選択されたポイントの情報を抽出
#     points_info = []
#     for point in selectedData['points']:
#         x = point['x']
#         y = point['y']
#         points_info.append(f"({x:.2f}, {y:.2f})")

#     # クラスタidを取得
#     selected_cluster_ids = get_clusters_from_points(
#         [int(point['pointIndex']) for point in selectedData['points']],
#         point_id_map
#     )
#     print(f"length of selected cluster ids: {len(selected_cluster_ids)}")

#     # dendrogram上のidを取得(node_id_mapから)
#     selected_new_cluster_ids = [node_id_map[k] for k in selected_cluster_ids if k in node_id_map]
#     print(f"length of selected new cluster ids: {len(selected_new_cluster_ids)}")

#     # dendrogramの色付けを変更
#     _Z, _ = filter_linkage_matrix_by_strahler(Z, S_min=strahler_threshold, N_leaves=len(_get_leaves(clusterer._raw_tree)))
#     print(f"Filtered Z shape: {_Z.shape}")
#     segments = get_dendrogram_segments2(_Z[:, [0, 1, 3, 4]])
#     # 3倍に複製
#     selected_new_cluster_ids = sum([[i]*3 for i in selected_new_cluster_ids], [])
#     fig = plot_dendrogram_plotly(segments, selected_indices=selected_new_cluster_ids)
#     return fig


# --- アプリケーションの実行 ---
# 'inline' モードでJupyter Notebook/Labのセル内に表示
app.run_server(mode='external', port=8051, host ='localhost')

<>:129: SyntaxWarning:

"is not" with a literal. Did you mean "!="?

<>:129: SyntaxWarning:

"is not" with a literal. Did you mean "!="?

C:\Users\acero\AppData\Local\Temp\ipykernel_26564\2629791921.py:129: SyntaxWarning:

"is not" with a literal. Did you mean "!="?



Dash is running on http://localhost:8051/

len segment, is_selected: 516, 0
Dash app running on http://localhost:8051/


length of selected cluster ids: 10
selected cluster ids: [100226, 100195, 100227, 100261, 100266, 100236, 100243, 100245, 100253, 100254]
length of selected new cluster ids: 10
selected new cluster ids: [45, 114, 46, 51, 53, 103, 48, 98, 100, 99]
len segment, is_selected: 516, 516
Selected cluster similarity matrix shape: (10, 10)
shape of selected cluster ids: 10
length of selected cluster ids: 97
selected cluster ids: [100002, 100003, 100006, 100008, 100010, 100015, 100017, 100020, 100021, 100022]
length of selected new cluster ids: 97
selected new cluster ids: [172, 0, 1, 2, 3, 4, 170, 167, 168, 169]
len segment, is_selected: 516, 516
Selected cluster similarity matrix shape: (97, 97)
shape of selected cluster ids: 97
length of selected cluster ids: 13
selected cluster ids: [100224, 100225, 100258, 100259, 100168, 100215, 100212, 100214, 100183, 100026]
length of selected new cluster ids: 13
selected new cluster ids: [43, 44, 101, 50, 119, 110, 108, 109, 35, 5]
len segment, is_selec

In [ ]:
# selected_cluster_ids = get_clusters_from_points(
#             [int(point['pointIndex']) for point in selectedData['points']],
#             point_id_map
#         )

selected_new_cluster_ids = [node_id_map[k] for k in selected_cluster_ids if k in node_id_map]

selected_new_cluster_ids = sum([[i]*3 for i in selected_new_cluster_ids], [])
text = f"選択されたデータポイントのクラスターid: {', '.join(map(str, selected_new_cluster_ids))}"

selected_cluster_ids = get_clusters_from_points([1, 2, 3, 99999, 939393], point_id_map)
print(selected_cluster_ids)
selected_new_cluster_ids = [node_id_map[k] for k in selected_cluster_ids if k in node_id_map]
print(selected_new_cluster_ids)

[100000, 100001, 100210]
[344, 343, 236]


In [111]:
# noise ratio
len([i for i in labels if i == -1]) / len(labels)
print(len(labels))
print(len(X))
print(len([[i] for i in labels]))

100000
100000
100000


In [37]:
print(len(node_id_map)) # フィルタ後のノード数
print(len(Z)) # 元のZ行列の行数
print(len(stability_dict)) # 元のノードIDに対応するstabilityスコ
print(len(node_id_map_reversed))
# print(N_leaves)
print(len(id_point_map))
print(len(point_id_map))

345
172
345
345
345
100000
